# Text Summarization. Homework

Всем привет! Это домашка по суммаризации текста.

На семинаре мы рассмотрели базовые модели для суммаризации текста. Попробуйте теперь улучшить два метода: TextRank и Extractive RNN. Задание достаточно большое и требует хорошую фантазию, тут можно эксперементировать во всю.

Для сдачи заданий надо получить определенное качество по test-у:

- 1 задание: 0.35 BLEU
- 2 задание: 0.35 BLEU

Если ваш подход пробивает это качество – задание считается пройденным. Плюсом будет описание того, почему вы решили использовать то или иное решение. 

Датасет: gazeta.ru

**P.S.** Возможно, в датасете находятся пустые данные. Проверьте эту гипотезу, и если надо, сделайте предобратоку датасета.


`Ноутбук создан на основе семинара Гусева Ильи на кафедре компьютерной лингвистики МФТИ.`

Загрузим датасет и необходимые библиотеки

In [1]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
!pip install -Uq razdel allennlp torch fasttext OpenNMT-py networkx pymorphy2 nltk rouge==0.3.1 summa
!pip install -Uq transformers youtokentome

     |████████████████████████████████| 512kB 6.0MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 204kB 11.7MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 1.4MB 12.8MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 266kB 21.0MB/s 
     |████████████████████████████████| 133kB 21.0MB/s 
     |████████████████████████████████| 1.3MB 19.4MB/s 
     |████████████████████████████████| 317kB 35.0MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 2.5MB 29.4MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 8.2MB 24.4MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 7.0MB 48.3MB/s 
     |████████████████████████████████| 2.9MB 62.9MB/s 
     |

In [ ]:
!head -n 1 gazeta_train.txt
!cat gazeta_train.txt | wc -l
!cat gazeta_val.txt | wc -l
!cat gazeta_test.txt | wc -l

{"url": "https://www.gazeta.ru/financial/2011/11/30/3852658.shtml", "text": "«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожида

In [ ]:
!head -n 10 gazeta_train.txt

{"url": "https://www.gazeta.ru/financial/2011/11/30/3852658.shtml", "text": "«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд. При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу» замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо: по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился: в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии. «После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожида

In [ ]:
!head -n 10 gazeta_test.txt

{"url": "https://www.gazeta.ru/science/2020/02/14_a_12960289.shtml", "text": "Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том, ка

In [3]:
import random
import pandas as pd

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [4]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [ ]:
test_col = test_records.columns
train_col = train_records.columns
val_col = val_records.columns
train_col
test_col

Index(['url', 'text', 'title', 'summary', 'date'], dtype='object')

In [ ]:
#train_records.loc[train_records['url']=='']
def empty_finder(col_list,dataframe_):
  for elem in col_list:
    print(elem)
    print(dataframe_.loc[dataframe_[elem]==''])


In [ ]:
empty_finder(train_col,train_records)
empty_finder(test_col,test_records)
empty_finder(val_col,val_records)

url
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
text
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
title
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
summary
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
date
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
url
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
text
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
title
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
summary
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
date
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
url
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
text
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
title
Empty DataFrame
Columns: [url, text, title, summary, date]
Index: []
summary
Empty DataFrame
Columns:

## 1 задание: TextRank (порог: 0.35 BLEU)

TextRank - unsupervised метод для составления кратких выжимок из текста. 
Описание метода:

1. Сплитим текст по предложениям
2. Считаем "похожесть" предложений между собой
3. Строим граф предложений с взвешенными ребрами
4. С помощью алгоритм PageRank получаем наиболее важные предложения, на основе которых делаем summary.

Функция похожести можно сделать и из нейросетевых(или около) моделек: FastText, ELMO и BERT. Выберете один метод, загрузите предобученную модель и с ее помощью для каждого предложениия сделайте sentence embedding. С помощью косинусной меры определяйте похожесть предложений.

Предобученные модели можно взять по [ссылке](http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html).

In [5]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [6]:
# texts = ['Привет,я хочу получить свои ебмедденги слов', 'И мои пожалуйста!']
# tokens,token_embs,subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = m(texts)
!pip install sentence_transformers

     |████████████████████████████████| 71kB 3.6MB/s 
  Using cached https://files.pythonhosted.org/packages/3a/83/e74092e7f24a08d751aa59b37a9fc572b2e4af3918cb66f7766c3affb1b4/transformers-3.5.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4c/34/b39eb9994bc3c999270b69c9eea40ecc6f0e97991dba28282b9fd32d44ee/tokenizers-0.9.3-cp36-cp36m-manylinux1_x86_64.whl
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=1e8916dd0b51210580c7192b091c68b595dfd1229fe08f52ac494bd566a502fa
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
Successfully built sentence-transformers
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.0.0
    Uninstalling transformers-4.0.0:
      Successfully uninstalled transformers-4.0.0


In [7]:
#Non Pavlov
from sentence_transformers import SentenceTransformer

In [8]:
#model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens',device='cuda') #low
model2 = SentenceTransformer('distilbert-base-nli-mean-tokens',device='cuda')
#model3 = SentenceTransformer(m,device='cuda')

100%|██████████| 245M/245M [00:11<00:00, 21.9MB/s]


In [9]:
from itertools import combinations
from numpy.linalg import norm
import networkx as nx
import numpy as np
import pymorphy2
import razdel
from sentence_transformers import SentenceTransformer, util
from numpy.linalg import norm



def your_super_words_similarity(words1, words2):
    # Your code
    #my_score = calc_scores(words1,words2)
    #print(my_score)
    #v1 = m([words1])
    #v2 = m([words2])
    #emb1 = model.encode([words1])
    #emb2 = model.encode([words2])
    #-norm(features1 - features2)
    #util.pytorch_cos_sim(emb1, emb2)
    #util.pytorch_cos_sim(words1, words2)
    return util.pytorch_cos_sim(words1, words2)


def gen_text_rank_summary(text, calc_similarity=your_super_words_similarity, summary_part=0.1, lower=True, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text.lower() for sentence in razdel.sentenize(text)]
    #sent_embed = model.encode(sentences)
    n_sentences = len(sentences)
    #print(sentences,'sentense words')
    #print(n_sentences,'n_sentences')



    # Токенизируем предложения
    #sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]
    #model_words2 = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]
    #sentences_words = [sentence.lower() for sentence in sentences]
    #print(sentences_words,'tokenized')

    #0.33
    model_words = [[token.lower() for token in model2.tokenizer.tokenize(sentence)] for sentence in sentences]
    model_words2 = [x if len(x)>1 else 'солнце коты идут' for x in model_words]


    # При необходимости лемматизируем слова
    if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]

    # Для каждой пары предложений считаем близость
    pairs = combinations(range(n_sentences), 2)

    #[print(i,j) for i,j in pairs]
    sent_word_embedings = model2.encode(model_words2,device='cuda')
    #pairs = combinations(range(n_sentences), 2)
    

    #scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]
    scores = [(i, j, calc_similarity(sent_word_embedings[i], sent_word_embedings[j])) for i, j in pairs]
    
    #print(len(scores),'scores')

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank(g,max_iter=60000000000)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    #predicted_summary = result[0]
    return predicted_summary

def calc_text_rank_score(records, calc_similarity=your_super_words_similarity, summary_part=0.088, lower=True, nrows=len(test_records), morph=None):
    references = []
    predictions = []
    print(len(records))
    conters = 0

    for text, summary in records[['text', 'summary']].values[:nrows]:
        conters += 1
        if conters%100==0:
          print(conters)

        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary = gen_text_rank_summary(text, calc_similarity, summary_part, lower, morph=None)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

In [ ]:
calc_text_rank_score(test_records, calc_similarity=your_super_words_similarity) #model2 - 1k 11:43- summary_part=0.088

5770
100
200
300
400
500
600
700
800
900
1000
Count: 1000
Ref: модернизация одной из самых мощных пушек в мире «пион», получившая название «малка», завершится до конца декабря. импортные части в орудиях заменили отечественными вариантами, сейчас пушка находится в поле и успешно проходит испытания.
Hyp: сейчас модернизированная «малка» находится в поле и успешно проходит испытания», — добавил семизоров. сау 2с7м «малка» является модернизированным вариантом орудия 2с7 «пион», созданного в 1970-х годах и до сих пор считающегося одной из самых мощных пушек в мире. вооруженным силам арабских государств было абсолютно нечего противопоставить самоходным пушкам м-107. самоходная пушка 2с7 выполнена по безбашенной схеме, с размещением орудия открыто в кормовой части гусеничного шасси.
BLEU:  0.3507615850423708
ROUGE:  {'rouge-1': {'f': 0.13964039733877442, 'p': 0.14766526430498575, 'r': 0.14452416288888603}, 'rouge-2': {'f': 0.0303848597611867, 'p': 0.03233316183487606, 'r': 0.03188029138801109

In [ ]:
calc_text_rank_score(test_records, calc_similarity=your_super_words_similarity) #model2 - total 14:36- summary_part=0.088

5770
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
Count: 5770
Ref: вашингтон не обладал точной информацией о планах иранского генерала касема сулеймани по нападению на сша, признал госсекретарь майк помпео. тем не менее, по его словам, у белого дома все равно не было сомнений относительно неизбежности атак не только на сша, но и на другие страны.
Hyp: думаю, нет сомнений, что сулеймани намеревался предпринять действия не только против наших сил и дипломатов в ираке», — указал помпео. иранский постпред при организации маджид тахт-раванчи заявил, что иран не может принять предложение вашингтона. иранские военные атаковали объекты сша, с которых было совершено «трусливое вооруженное нападение», уточнил дипломат, тегеран не стремится к войне, но будет защищать се

## 2 Задание: Extractive RNN (порог: 0.35 BLEU)

Второй метод, который вам предлагается улучшить – поиск предложений для summary с помощью RNN. В рассмотренной методе мы использовали LSTM для генерации sentence embedding. Попробуйте использовать другие архитектуры: CNN, Transformer; или добавьте предобученные модели, как и в первом задании.

P.S. Тут предполагается, что придется изменять много кода в ячееках (например, поменять токенизацию). 

### Модель

Картинка для привлечения внимания:

![img](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_14%2Fproject_398421%2Fimages%2Farchitecture.png)

Статья с оригинальным методом:
https://arxiv.org/pdf/1611.04230.pdf

Список вдохновения: 
- https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b Пример того, как можно применять CNN в текстовых задачах
- https://arxiv.org/pdf/1808.08745.pdf Очень крутой метод генерации summary без Transformers
- https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c – простой метод генерации sentence embedding
- https://towardsdatascience.com/fse-2b1ffa791cf9 – Необычный метод генерации sentence embedding
- https://github.com/UKPLab/sentence-transformers – BERT предобученный для sentence embedding

P.S. Выше написанные ссылки нужны только для разогрева вашей фантазии, можно воспользоваться ими, а можно придумать свой.

Комментарий к заданию:
Если посмотреть на архитектуру ~~почти~~ SummaRuNNer, то в ней есть два главных элемента: первая часть, которая читает предложения и возвращает векторы на каждое предложение, и вторая, которая выбирает предложения для суммаризации. Вторую часть мы не трогаем, а первую меняем. На что меняем – как вы решите. Главное: она должна иметь хорошее качество и встроиться в текущую модель.

In [46]:
import copy
import random

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    score = -1.0
    summaries = []
    for _ in range(min(n_sentences, 2)):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [11]:
from tqdm import tqdm_notebook as tqdm

def calc_oracle_score(records, nrows=1000, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for text, summary in tqdm(records[['text', 'summary']].values[:nrows]):
        summary = summary if not lower else summary.lower()
        references.append(summary)
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_oracle_score(test_records)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Count: 1000
Ref: команда владимира зеленского и партия «слуга народа» уничтожат украинскую нацию, заявил депутат верховной рады илья кива. по его мнению, президент украины действительно хочет принести мир, но с такими сторонниками ему никаких врагов не надо — правящая партия «делает колонию» из страны. он также подчеркнул, что сподвижники неопытного зеленского «заведут его в блуд».
Hyp: дальнейшая политика команды президента украины владимира зеленского и его партии «слуга народа» грозит уничтожением украинской нации, уверен депутат верховной рады от партии «оппозиционная платформа — за жизнь» илья кива. он также подчеркнул, что под разговоры о «российской агрессии» интересы украины сдают партнерам с запада и из сша.
BLEU:  0.42980751362196556
ROUGE:  {'rouge-1': {'f': 0.3543253390523239, 'p': 0.4321182006506703, 'r': 0.3179937018393389}, 'rouge-2': {'f': 0.2030358748946701, 'p': 0.25423914875929554, 'r': 0.18090410188798609}, 'rouge-l': {'f': 0.3050319991365936, 'p': 0.40243012346476

## (!)
Если надо, поменяйте код загрузки токенизатора

In [12]:
!pip install youtokentome

In [13]:
import os

import youtokentome as yttm

def train_bpe(records, model_path, model_type="bpe", vocab_size=30000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "w") as temp:
        for text, summary in records[['text', 'summary']].values:
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write(text + "\n")
            temp.write(summary + "\n")
    yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

train_bpe(train_records, "BPE_model.bin")


In [14]:
bpe_processor = yttm.BPE('BPE_model.bin')
bpe_processor.encode(["октябрь богат на изменения"], output_type=yttm.OutputType.SUBWORD)

[['▁октябрь', '▁богат', '▁на', '▁изменения']]

## (!)
Если надо, поменяйте код словаря

In [15]:
from collections import Counter
from typing import List, Tuple
import os

class Vocabulary:
    def __init__(self, bpe_processor):
        self.index2word = bpe_processor.vocab()
        self.word2index = {w: i for i, w in enumerate(self.index2word)}
        self.word2count = Counter()

    def get_pad(self):
        return self.word2index["<PAD>"]

    def get_sos(self):
        return self.word2index["<SOS>"]

    def get_eos(self):
        return self.word2index["<EOS>"]

    def get_unk(self):
        return self.word2index["<UNK>"]
    
    def has_word(self, word) -> bool:
        return word in self.word2index

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        return self.get_unk()

    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        empty_size = 4
        return self.size() <= empty_size

    def reset(self):
        self.word2count = Counter()
        self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
        self.word2index = {word: index for index, word in enumerate(self.index2word)}

In [16]:
vocabulary = Vocabulary(bpe_processor)
vocabulary.size()

30000

In [17]:
from rouge import Rouge
import razdel

def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1000):
    rouge = Rouge()
    sentences_ = []
    oracle_sentences_ = []
    oracle_summary_ = []
    if nrows is not None:
        records = records.iloc[:nrows].copy()
    else:
        records = records.copy()

    for text, summary in tqdm(records[['text', 'summary']].values):
        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        sentences_ += [sentences]
        oracle_sentences_ += [list(sentences_indicies)]
        oracle_summary_ += [oracle_summary]
    records['sentences'] = sentences_
    records['oracle_sentences'] = oracle_sentences_
    records['oracle_summary'] = oracle_summary_
    return records

#ext_train_records = add_oracle_summary_to_records(train_records, nrows=30000)
#ext_val_records = add_oracle_summary_to_records(val_records, nrows=None)
#ext_test_records = add_oracle_summary_to_records(test_records, nrows=None)

Используй `pickle` для сохранения записей, чтобы потом не пересоздавать их потом. Если решаешь задание в колабе, можешь подключить свой гугл диск и сохранить данные в нём.

In [18]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

pickle_train = open("/content/drive/MyDrive/summar_2020/train_records.bin","rb")
pickle_test = open("/content/drive/MyDrive/summar_2020/test_records.bin","rb")
pickle_valid = open("/content/drive/MyDrive/summar_2020/val_records.bin","rb")

ext_test_records  = pickle.load(pickle_test)
ext_val_records = pickle.load(pickle_valid)
ext_train_records = pickle.load(pickle_train)


Mounted at /content/drive


## (!)
Если надо, поменяйте код генератора датасета и батчевалки

In [19]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


from torch.utils import data


class ExtDataset(data.Dataset):
    def __init__(self, records, vocabulary, bpe_processor, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.bpe_processor = bpe_processor
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.records.shape[0]

    def __getitem__(self, idx):
        cur_record = self.records.iloc[idx]
        inputs = list(map(lambda x: x[:self.max_sentence_length], self.bpe_processor.encode(cur_record['sentences'], output_type=yttm.OutputType.ID)))
        outputs = [int(i in cur_record['oracle_sentences']) for i in range(len(cur_record['sentences']))]
        return {'inputs': inputs, 'outputs': outputs}

In [20]:
# Это батчевалка
def collate_fn(records):
    max_length = max(len(sentence) for record in records for sentence in record['inputs'])
    max_sentences = max(len(record['outputs']) for record in records)

    new_inputs = torch.zeros((len(records), max_sentences, max_length))
    new_outputs = torch.zeros((len(records), max_sentences))
    for i, record in enumerate(records):
        for j, sentence in enumerate(record['inputs']):
            new_inputs[i, j, :len(sentence)] += np.array(sentence)
        new_outputs[i, :len(record['outputs'])] += np.array(record['outputs'])
    return {'features': new_inputs.type(torch.LongTensor), 'targets': new_outputs}

In [21]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack


class SentenceEncoder(nn.Module):
    # Место для вашего Sentence Encoder-а. Разрешается использовать любые методы, которые вам нравятся.
    def __init__(self,input_size,embedding_dim,hidden_size,n_layers=3,dropout=0.3,bidirectional=True):
      super().__init__()
      num_directions = 2 if bidirectional else 1
      assert hidden_size % num_directions == 0
      hidden_size = hidden_size // num_directions
      self.embedding_dim = embedding_dim
      self.input_size = input_size
      self.hidden_size = hidden_size
      self.n_layers = n_layers
      self.dropout = dropout
      self.bidirectional = bidirectional

      self.embedding_layer = nn.Embedding(input_size,embedding_dim)
      self.rnn_layer = nn.LSTM(embedding_dim,hidden_size,
                               n_layers,dropout=dropout,
                               bidirectional=bidirectional,batch_first=True)
      self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
      embedded = self.embedding_layer(inputs)
      outputs,_ = self.rnn_layer(embedded,hidden)
      sentences_embeddings = torch.mean(outputs,1)
      return sentences_embeddings



class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=3,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        # Your sentence encoder model
        self.sentence_encoder = SentenceEncoder(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)
        
        self.rnn_layer = nn.LSTM(
            sentence_encoder_hidden_size, 
            hidden_size, 
            n_layers, 
            dropout=dropout,
            bidirectional=bidirectional, 
            batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        content = self.content_linear_layer(outputs).squeeze(2)
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        return content + salience

model = SentenceTaggerRNN(vocabulary.size())

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
# with open("val_records.bin", 'wb') as file:
#     pickle.dump(ext_val_records,file,)

# pickle_train = open("/content/drive/MyDrive/summar_2020/train_records.bin","rb")
# pickle_test = open("/content/drive/MyDrive/summar_2020/test_records.bin","rb")
# pickle_valid = open("/content/drive/MyDrive/summar_2020/val_records.bin","rb")

# ext_test_records  = pickle.load(pickle_test)
# ext_val_records = pickle.load(pickle_valid)
# ext_train_records = pickle.load(pickle_train)

### Обучение

In [81]:
device = torch.device('cuda')

loaders = {
    'train': data.DataLoader(
        ExtDataset(
            ext_train_records, 
            vocabulary, 
            bpe_processor=bpe_processor
        ), 
        batch_size=128, 
        collate_fn=collate_fn
    ),
    'valid': data.DataLoader(
        ExtDataset(
            ext_val_records, 
            vocabulary, 
            bpe_processor=bpe_processor
        ), 
        batch_size=128, 
        collate_fn=collate_fn
    ),
    'test': data.DataLoader(
        ExtDataset(
            ext_test_records, 
            vocabulary, 
            bpe_processor=bpe_processor
        ), 
        batch_size=64, 
        collate_fn=collate_fn
    ),
}

lr = 12e-5
num_epochs = 1

optimizer  = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()
# Maybe adding scheduler? #8
#тренировка модели всего 8 эпох.
#6 эпох с рейтом 10-4, 7-8 эпоха 15*10-5

In [82]:
from tqdm.notebook import trange, tqdm


def train():
    model.to(device)
    pbar_loader = trange(len(loaders["train"]) + len(loaders["valid"]), desc=f"Train Loss: {0}, Valid Loss: {0}")
    for e in trange(num_epochs, desc="Epoch"):
        train_loss = 0
        valid_loss = 0
        train_it = 0
        valid_it = 0
        
        model.train()
        for batch in loaders["train"]:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            
            logits = model(features)
            
            loss = criterion(logits, targets)
            train_loss += loss.item()
            train_it += 1
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Maybe adding scheduler?
            
            pbar_loader.update()
            pbar_loader.set_description(
                f"Train Loss: {train_loss / train_it:.3}, Valid Loss: {0}"
            )
            
        model.eval()
        with torch.no_grad():
            for batch in loaders["valid"]:
                features = batch["features"].to(device)
                targets = batch["targets"].to(device)

                logits = model(features)

                loss = criterion(logits, targets)
                valid_loss += loss.item()
                valid_it += 1
                
                pbar_loader.update()
                pbar_loader.set_description(
                    f"Train Loss: {train_loss / train_it:.3},"
                    f" Valid Loss: {valid_loss / valid_it:.3}"
                )
        print(
            f"Epoch {e}; Train Loss: {train_loss / train_it:.3},"
            f" Valid Loss: {valid_loss / valid_it:.3}"
        )
        pbar_loader.reset()

In [83]:
train()

Epoch 0; Train Loss: 0.173, Valid Loss: 0.176



In [89]:
device = torch.device("cuda")
import re
def punct_detokenize(text):
    text = text.strip()
    punctuation = ",.!?:;%"
    closing_punctuation = ")]}"
    opening_punctuation = "([}"
    for ch in punctuation + closing_punctuation:
        text = text.replace(" " + ch, ch)
    for ch in opening_punctuation:
        text = text.replace(ch + " ", ch)
    res = [r'"\s[^"]+\s"', r"'\s[^']+\s'"]
    for r in res:
        for f in re.findall(r, text, re.U):
            text = text.replace(f, f[0] + f[2:-2] + f[-1])
    text = text.replace("' s", "'s").replace(" 's", "'s")
    text = text.strip()
    return text

def postprocess(ref, hyp, is_multiple_ref=False, detokenize_after=False, tokenize_after=True):
    if is_multiple_ref:
        reference_sents = ref.split(" s_s ")
        decoded_sents = hyp.split("s_s")
        hyp = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in decoded_sents]
        ref = [w.replace("<", "&lt;").replace(">", "&gt;").strip() for w in reference_sents]
        hyp = " ".join(hyp)
        ref = " ".join(ref)
    ref = ref.strip()
    hyp = hyp.strip()
    if detokenize_after:
        hyp = punct_detokenize(hyp)
        ref = punct_detokenize(ref)
    if tokenize_after:
        hyp = hyp.replace("@@UNKNOWN@@", "<unk>")
        hyp = " ".join([token.text for token in razdel.tokenize(hyp)])
        ref = " ".join([token.text for token in razdel.tokenize(ref)])
    return ref, hyp    

references = []
predictions = []
model.eval()

for num, item in tqdm(enumerate(loaders["test"]), total=len(loaders["test"])):
    logits = model(item["features"].to(device))[0]
    log_sort = torch.argsort(logits, dim=0)[-3:]
    
    record = ext_test_records.iloc[num]
    predicted_summary = []
    #print(item['targets'].shape,'target')
    #print(logits,'logits')
    
    # for i, logit in enumerate(logits):
    #     if logit > 0.0:
    #         print('In!')
    #         predicted_summary.append(record['sentences'][i])
        

    # if not predicted_summary:
    #    predicted_summary.append(record['sentences'][log_sort[-1].item()])
    predicted_summary.append(record['sentences'][log_sort[-1].item()])
    predicted_summary.append(record['sentences'][log_sort[-2].item()])
    predicted_summary.append(record['sentences'][log_sort[-3].item()])

    predicted_summary = " ".join(predicted_summary)
    summary, predicted_summary= postprocess(record['summary'], predicted_summary)
    
    references.append(summary)
    predictions.append(predicted_summary)

calc_scores(references, predictions)


Count: 91
Ref: У детей , рожденных из замороженных эмбрионов , риск развития рака в 2,4 раза выше , чем у тех , кто был зачат естественным путем , выяснили датские специалисты . Самые частые его виды — лейкемия и опухоли нервной системы . Однако относительный риск все равно невелик , успокаивают исследователи .
Hyp: заморозка эмбрионов перед эко повышает риск развития рака у детей в 2,4 раза , предупреждают исследователи из датского исследовательского центра общества борьбы с раком . чаще всего — в трети случаев — это была лейкемия . предположительно , заморозка влияет на процессы внутриутробного роста , и , возможно , именно они в итоге повышают риск развития рака .
BLEU:  0.40782866856104727
ROUGE:  {'rouge-1': {'f': 0.22833328703226322, 'p': 0.22227919501914958, 'r': 0.2498889579265903}, 'rouge-2': {'f': 0.07757583606699943, 'p': 0.07439124853358245, 'r': 0.08682461962409739}, 'rouge-l': {'f': 0.1964914917409234, 'p': 0.20161242681721261, 'r': 0.22503385889361047}}
